In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TruePositives,FalsePositives,TrueNegatives,FalseNegatives,Precision,Recall,AUC
from PIL import Image
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
import seaborn as sns
import io

2024-02-15 13:47:25.652287: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 13:47:25.652349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 13:47:25.653789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
img_path='/kaggle/input/plant-pathology-2020-fgvc7/images/'

### Configurations: ALL Hyperparameters I am willing to change

In [3]:
CONFIG={'BATCH_SIZE':64,
        'IMG_SIZE':256,
        'LR':0.0001,
        'VAL_SIZE':0.15,
        'EPOCHS':20,
        'OPTIMIZER':'adam',
        'LOSS':'categorical_crossentropy',
        'CONV_1':32,
        'CONV_2':64,
        'CONV_3':128,
        'CONV_4':256,
        'DROPOUT':0.3,
        'DENSE_1':512,
        'MAX_POOLING':(2,2),
        'CLASS_NAMES':['Healthy','Multiple Diseases','Rust','Scab']
       }

In [4]:
train=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')
train.head()


,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [5]:
def paths(x):
    return img_path+ x +'.jpg'
    

In [6]:
train_paths = train.image_id.apply(paths).values
test_paths=test.image_id.apply(paths).values
train_paths

array(['/kaggle/input/plant-pathology-2020-fgvc7/images/Train_0.jpg',
       '/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1.jpg',
       '/kaggle/input/plant-pathology-2020-fgvc7/images/Train_2.jpg', ...,
       '/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1818.jpg',
       '/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1819.jpg',
       '/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1820.jpg'],
      dtype=object)

In [7]:
train_labels = train.loc[:, 'healthy':].values
train_labels

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [8]:
train_paths, val_paths, train_labels, val_labels =train_test_split(train_paths, train_labels, test_size=CONFIG['VAL_SIZE'], random_state=42)
print("Train:",len(train_paths))
print('Val:',len(val_paths))

Train: 1547
Val: 274


In [9]:
def preprocess(image_path,label=None):
    bits=tf.io.read_file(image_path)
    image=tf.io.decode_jpeg(bits,channels=3)
    
    image=tf.cast(image,tf.float32)/255.0
    
    image=tf.image.resize(image,(CONFIG['IMG_SIZE'],CONFIG['IMG_SIZE']))
    
    
    if label is None:
        return image
    else:
        return image,label

In [10]:
def augment(image,label=None):
    image=tf.image.random_flip_left_right(image)
    image=tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image,label

    

In [11]:
train_dataset=tf.data.Dataset.from_tensor_slices((train_paths,train_labels))
train_dataset=(train_dataset
                   .map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
                   .cache()
                   .repeat(2)
                   .map(augment,num_parallel_calls=tf.data.experimental.AUTOTUNE)
                   .shuffle(512)
                   .batch(CONFIG['BATCH_SIZE'])
                   .prefetch(tf.data.experimental.AUTOTUNE)
                  )

print('Train:',len(train_dataset))

Train: 49


In [12]:
val_dataset=tf.data.Dataset.from_tensor_slices((val_paths,val_labels))
val_dataset=(val_dataset
                .map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
               .cache()
               .shuffle(512)
               .batch(CONFIG['BATCH_SIZE'])
               .prefetch(tf.data.experimental.AUTOTUNE)
        )
print('Val:',len(val_dataset))

Val: 5


In [13]:
test_dataset=tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset=(test_dataset
                 .map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 .map(augment,num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 .batch(CONFIG['BATCH_SIZE'])
             )

In [14]:
def create_model(input_shape,num_classes):
    model=models.Sequential()
    
    model.add(layers.Conv2D(CONFIG['CONV_1'],(3,3),activation='relu',input_shape=input_shape))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))

    model.add(layers.Conv2D(CONFIG['CONV_2'], (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))

    model.add(layers.Conv2D(CONFIG['CONV_3'], (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))

    model.add(layers.Conv2D(CONFIG['CONV_4'], (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))
              
        
    model.add(layers.Dropout(CONFIG['DROPOUT']))
    model.add(layers.Flatten())
    
    model.add(layers.Dense(CONFIG['DENSE_1'], activation='relu'))
              
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [15]:
input_shape = (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 3)
num_classes = 4

model = create_model(input_shape, num_classes)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [16]:
model.compile(
        optimizer = CONFIG['OPTIMIZER'],
        loss = CONFIG['LOSS'],
        metrics=['categorical_accuracy']
    )


In [17]:
history = model.fit(
    train_dataset, 
    epochs=CONFIG['EPOCHS'],
    validation_data=val_dataset
)

Epoch 1/20


2024-02-15 13:47:33.907845: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1708004863.589167    1424 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 [==============================] - 35s 402ms/step - loss: 1.3011 - categorical_accuracy: 0.3271 - val_loss: 1.2294 - val_categorical_accuracy: 0.3540
Epoch 2/20
49/49 [==============================] - 11s 221ms/step - loss: 1.0563 - categorical_accuracy: 0.5488 - val_loss: 0.7977 - val_categorical_accuracy: 0.6898
Epoch 3/20
49/49 [==============================] - 11s 221ms/step - loss: 0.7482 - categorical_accuracy: 0.7162 - val_loss: 0.6611 - val_categorical_accuracy: 0.7190
Epoch 4/20
49/49 [==============================] - 11s 220ms/step - loss: 0.5976 - categorical_accuracy: 0.7553 - val_loss: 0.6365 - val_categorical_accuracy: 0.7226
Epoch 5/20
49/49 [==============================] - 11s 222ms/step - loss: 0.5204 - categorical_accuracy: 0.7983 - val_loss: 0.6109 - val_categorical_accuracy: 0.7555
Epoch 6/20
49/49 [==============================] - 11s 220ms/step - loss: 0.4765 - categorical_accuracy: 0.8132 - val_loss: 0.5468 - val_categorical_accuracy: 0.7810
Epoch 7/2

# WandB

In [18]:
pip install wandb --upgrade

Note: you may need to restart the kernel to use updated packages.


In [19]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

wandb: Currently logged in as: kareem8996. Use `wandb login --relogin` to force relogin


True

In [20]:
wandb.init(project='Plant-Pathology')

In [21]:
wandb.config=CONFIG

## Metrics

In [22]:
metrics=[TruePositives(name='tp'),FalsePositives(name='fp'),TrueNegatives(name='tn'),FalseNegatives(name='fn'),Precision(name='precision'),Recall(name='recall'),AUC(name='auc'),'categorical_accuracy']
val_features, val_labels = next(iter(val_dataset))

## Confusion Matrix

#### Image from SNS

In [23]:
class LogImagesCallbackWandB(Callback):
  def on_epoch_end(self, epoch, logs):
    labels = val_labels
    inp = val_features
    
    labels = np.array([np.argmax(i) for i in labels])
    predicted = model.predict(inp)
    predicted=np.array([np.argmax(i) for i in predicted])


    cm = confusion_matrix(labels,predicted)
    
    plt.figure(figsize=(8,8))

    sns.heatmap(cm, annot=True,)
    plt.title('Confusion matrix - {}'.format(0.5))
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.axis('off')

    buffer = io.BytesIO()
    plt.savefig(buffer, format = 'png')

    image_array = tf.image.decode_png(buffer.getvalue(), channels=3)

    images = wandb.Image(image_array, caption="Confusion Matrix for epoch: {}".format(epoch))
          
    wandb.log(
        {"Confusion Matrix": images})

#### OR Directly from WandB

In [24]:
class LogConfMatrix(Callback):
  def on_epoch_end(self, epoch, logs):
    labels = val_labels
    inp = val_features
    
    labels = np.array([np.argmax(i) for i in labels])
    predicted = model.predict(inp)
    predicted=np.array([np.argmax(i) for i in predicted])
    
    cm = wandb.plot.confusion_matrix(
        y_true=labels,
        preds=predicted,
        class_names=CONFIG["CLASS_NAMES"])
        
    wandb.log({"conf_mat": cm})

## Logging Results on Tables

In [25]:
class LogResultsTable(Callback):
  def on_epoch_end(self, epoch, logs):
    
    columns=["image", "Predicted", "Label"]
    
    val_table = wandb.Table(columns = columns)

    
    for im, label in val_dataset.take(25):

      pred = CONFIG["CLASS_NAMES"][tf.argmax(model(im), axis = -1).numpy()[0]]
      label = CONFIG["CLASS_NAMES"][tf.argmax(label, axis = -1).numpy()[0]]

      row = [wandb.Image(im[0]), pred, label]
      
      val_table.add_data(*row)

      
    wandb.log({"Model Results" : val_table})

### Training

In [26]:
model.compile(
        optimizer = CONFIG['OPTIMIZER'],
        loss = CONFIG['LOSS'],
        metrics=metrics
    )

In [27]:
history = model.fit(
    train_dataset, 
    epochs=CONFIG['EPOCHS'],
    validation_data=(val_features, val_labels),
    callbacks=[WandbCallback(validation_data=(val_features, val_labels),
                            labels=CONFIG['CLASS_NAMES'],
                            data_type='image'
                            ),
              LogConfMatrix(),
              LogResultsTable(),
              ]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/20


2024-02-15 13:52:46.802785: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2/2 [==============================] - 0s 321ms/steposs: 0.0831 - tp: 3014.0000 - fp: 75.0000 - tn: 9207.0000 - fn: 80.0000 - precision: 0.9757 - recall: 0.9741 - auc: 0.9976 - categorical_accuracy: 0.


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/kaggle/working/wandb/run-20240215_135152-thspdwgy/files/model-best)... Done. 1.3s


49/49 [==============================] - 28s 505ms/step - loss: 0.0831 - tp: 3014.0000 - fp: 75.0000 - tn: 9207.0000 - fn: 80.0000 - precision: 0.9757 - recall: 0.9741 - auc: 0.9976 - categorical_accuracy: 0.9741 - val_loss: 0.8255 - val_tp: 54.0000 - val_fp: 10.0000 - val_tn: 182.0000 - val_fn: 10.0000 - val_precision: 0.8438 - val_recall: 0.8438 - val_auc: 0.9624 - val_categorical_accuracy: 0.8438
Epoch 2/20
2/2 [==============================] - 0s 14ms/steploss: 0.0595 - tp: 3030.0000 - fp: 57.0000 - tn: 9225.0000 - fn: 64.0000 - precision: 0.9815 - recall: 0.9793 - auc: 0.9990 - categorical_accuracy: 0.98


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/kaggle/working/wandb/run-20240215_135152-thspdwgy/files/model-best)... Done. 1.2s


49/49 [==============================] - 19s 392ms/step - loss: 0.0595 - tp: 3030.0000 - fp: 57.0000 - tn: 9225.0000 - fn: 64.0000 - precision: 0.9815 - recall: 0.9793 - auc: 0.9990 - categorical_accuracy: 0.9806 - val_loss: 0.3425 - val_tp: 60.0000 - val_fp: 4.0000 - val_tn: 188.0000 - val_fn: 4.0000 - val_precision: 0.9375 - val_recall: 0.9375 - val_auc: 0.9829 - val_categorical_accuracy: 0.9375
Epoch 3/20
49/49 [==============================] - 14s 274ms/step - loss: 0.0296 - tp: 3062.0000 - fp: 27.0000 - tn: 9255.0000 - fn: 32.0000 - precision: 0.9913 - recall: 0.9897 - auc: 0.9998 - categorical_accuracy: 0.9900 - val_loss: 0.4128 - val_tp: 58.0000 - val_fp: 5.0000 - val_tn: 187.0000 - val_fn: 6.0000 - val_precision: 0.9206 - val_recall: 0.9062 - val_auc: 0.9840 - val_categorical_accuracy: 0.9062
Epoch 4/20
49/49 [==============================] - 13s 272ms/step - loss: 0.0459 - tp: 3055.0000 - fp: 38.0000 - tn: 9244.0000 - fn: 39.0000 - precision: 0.9877 - recall: 0.9874 - auc: 0

## Hyperparamter Tuning

### Grid

In [28]:
sweep_config = {
  "name" : "Plant-Pathology-Sweep",
  "method" : "random",
  "metric": {
      "name" : "categorical_accuracy",
      "goal" : "maximize",
  },
    
  "parameters" : {
    
      "IMG_SIZE": {
        "value" : 256
    },
      
      "CONV_1": {
        "values" : [8,16,32],
    },
      
      "CONV_2": {
        "values" : [32,64,128],
    },
      
      "CONV_3": {
        "values" : [128,256],
    },
      
      "CONV_4": {
        "values" : [256,512],
    },
      
      'MAX_POOLING':{
          'values':[(2,2),(3,3)]
      },
      
  
    "DENSE_1" : {
      "values" : [16, 32, 64, 128,256]
    },

    "DROPOUT":{
      "min": 0.1,
      "max": 0.4
    },
      
      "LEARNING_RATE" :{
      "distribution": "uniform",
      "min": 1e-4,
      "max": 1e-2
    }

  }
}
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: sp5mcvgi
Sweep URL: https://wandb.ai/kareem8996/uncategorized/sweeps/sp5mcvgi


### Model

In [29]:
def model_tune(CONFIG):
    model=models.Sequential()
    
    model.add(layers.Conv2D(CONFIG['CONV_1'],(3,3),activation='relu',input_shape=(CONFIG['IMG_SIZE'],CONFIG['IMG_SIZE'],3)))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))

    model.add(layers.Conv2D(CONFIG['CONV_2'], (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))

    model.add(layers.Conv2D(CONFIG['CONV_3'], (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))

    model.add(layers.Conv2D(CONFIG['CONV_4'], (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(CONFIG['MAX_POOLING']))
              
    model.add(layers.Dropout(CONFIG['DROPOUT']))
    model.add(layers.Flatten())
    model.add(layers.Dropout(CONFIG['DROPOUT']))

    model.add(layers.Dense(CONFIG['DENSE_1'], activation='relu'))
              
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [30]:
val_features, val_labels = next(iter(val_dataset))
train_features, train_labels = next(iter(train_dataset))

def train():
    with wandb.init(project="Plant-Pathology") as run:
        CONFIG = wandb.config
        model = model_tune(CONFIG)
        model.compile(
              optimizer= Adam(CONFIG['LEARNING_RATE']),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'],
              )
        model.fit(train_dataset, epochs=10, callbacks = [WandbCallback()])

count = 10 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: 8tgz1hfl with config:
wandb: 	CONV_1: 16
wandb: 	CONV_2: 64
wandb: 	CONV_3: 256
wandb: 	CONV_4: 512
wandb: 	DENSE_1: 128
wandb: 	DROPOUT: 0.35269756464844615
wandb: 	IMG_SIZE: 256
wandb: 	LEARNING_RATE: 0.0023424304478487624
wandb: 	MAX_POOLING: [2, 2]
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/10


2024-02-15 13:58:39.167111: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


...

2024-02-15 14:19:58.624213: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


49/49 [==============================] - 8s 128ms/step - loss: 1.2943 - categorical_accuracy: 0.3355
Epoch 2/10
49/49 [==============================] - 6s 128ms/step - loss: 1.2052 - categorical_accuracy: 0.4037
Epoch 3/10
49/49 [==============================] - 6s 128ms/step - loss: 1.1067 - categorical_accuracy: 0.5304
Epoch 4/10
49/49 [==============================] - 6s 129ms/step - loss: 0.8825 - categorical_accuracy: 0.6700
Epoch 5/10
49/49 [==============================] - 6s 129ms/step - loss: 0.6807 - categorical_accuracy: 0.7450
Epoch 6/10
49/49 [==============================] - 6s 130ms/step - loss: 0.5627 - categorical_accuracy: 0.7928
Epoch 7/10
49/49 [==============================] - 6s 129ms/step - loss: 0.4731 - categorical_accuracy: 0.8184
Epoch 8/10
49/49 [==============================] - 6s 128ms/step - loss: 0.4030 - categorical_accuracy: 0.8429
Epoch 9/10
49/49 [==============================] - 6s 129ms/step - loss: 0.3596 - categorical_accuracy: 0.8633
Epo

categorical_accuracy,▁▂▄▅▆▇▇███
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇▇▅▄▃▂▂▁▁
categorical_accuracy,0.87783
epoch,9
loss,0.32306


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fabf01c47f0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fabdaa4d7b0, execution_count=30 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fabdaa4c340, raw_cell="val_features, val_labels = next(iter(val_dataset)).." store_history=True silent=False shell_futures=True cell_id=0166b726-302b-4f61-a5dc-06e11d5682e3> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe